In [ ]:
from __future__ import division
import pyomo.environ as pyomo
import pyomo.opt as opt
import pandas as pd

# Welcome to the TESA programming exercise on basic dispatch LP models

## 1. Prepare the input data for the LP model

In [ ]:
# introduce sets of indices
T = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23] # set of indices for supply
G = ['CCGT','GT','Oil','Hydro','Coal','Lignite','Nuclear','Wind'] # generator technologies

In [ ]:
# constant price for CO2 emissions [€/t CO2]
CO2_PRICE = 20

In [ ]:
# convention:   tech_data[(g,'capacity')] = installed capacity of technology g [MW]
#               tech_data[(g,'eta_el')] = eta_electric of technology g [1]
#               tech_data[(g,'fuel price')]= fuel price of technology g [€/MWh_thermal]
#               tech_data[(g,'other variable costs')] = other variable costs of technology g [€/MWh_el]
#               tech_data[(g,'emission factor')] = emission factor of technology g [t CO2/MWh_thermal]
tech_data_g = {('CCGT','capacity'):     15991,  ('CCGT','eta_el'):      0.54,   ('CCGT','fuel price'):      23.4,   ('CCGT','other variable costs'):    1.5,    ('CCGT','emission factor'):     0.2048, 
               ('GT','capacity'):       953,    ('GT','eta_el'):        0.28,   ('GT','fuel price'):        23.4,   ('GT','other variable costs'):      1.5,    ('GT','emission factor'):       0.2048, 
               ('Oil','capacity'):      1893,   ('Oil','eta_el'):       0.28,   ('Oil','fuel price'):       50.1,   ('Oil','other variable costs'):     1.7,    ('Oil','emission factor'):      0.2664, 
               ('Hydro','capacity'):    5322,   ('Hydro','eta_el'):     1.0,    ('Hydro','fuel price'):     0.0,    ('Hydro','other variable costs'):   1.5,    ('Hydro','emission factor'):    0.0,
               ('Coal','capacity'):     29403,  ('Coal','eta_el'):      0.36,   ('Coal','fuel price'):      14.9,   ('Coal','other variable costs'):    2.6,    ('Coal','emission factor'):     0.342, 
               ('Lignite','capacity'):  20763,  ('Lignite','eta_el'):   0.38,   ('Lignite','fuel price'):   3.8,    ('Lignite','other variable costs'): 3.0,    ('Lignite','emission factor'):  0.3996, 
               ('Nuclear','capacity'):  20339,  ('Nuclear','eta_el'):   0.33,   ('Nuclear','fuel price'):   1.8,    ('Nuclear','other variable costs'): 0.7,    ('Nuclear','emission factor'):  0.0,
               ('Wind','capacity'):     10428,  ('Wind','eta_el'):      1.0,    ('Wind','fuel price'):      0.0,    ('Wind','other variable costs'):    1.5,    ('Wind','emission factor'):     0.0}

In [ ]:
# convention:   availability_g_t[g][t] = available capacity of technology g during hour t [MW]
availability_g_t = {'CCGT':     [14629,14629,14629,14629,14629,14629,14629,14629,14629,14629,14629,14629,14629,14629,14629,14629,14629,14629,14629,14629,14629,14629,14629,14629],
                    'GT':       [769,769,769,769,769,769,769,769,769,769,769,769,769,769,769,769,769,769,769,769,769,769,769,769,], 
                    'Oil':      [1717,1717,1717,1717,1717,1717,1717,1717,1717,1717,1717,1717,1717,1717,1717,1717,1717,1717,1717,1717,1717,1717,1717,1717], 
                    'Hydro':    [3963,3963,3963,3963,3963,3963,3963,3963,3963,3963,3963,3963,3963,3963,3963,3963,3963,3963,3963,3963,3963,3963,3963,3963], 
                    'Coal':     [23185,23185,23185,23185,23185,23185,23185,23185,23185,23185,23185,23185,23185,23185,23185,23185,23185,23185,23185,23185,23185,23185,23185,23185], 
                    'Lignite':  [18687,18687,18687,18687,18687,18687,18687,18687,18687,18687,18687,18687,18687,18687,18687,18687,18687,18687,18687,18687,18687,18687,18687,18687,], 
                    'Nuclear':  [20306,20306,20306,20306,20306,20306,20306,20306,20306,20306,20306,20306,20306,20306,20306,20306,20306,20306,20306,20306,20306,20306,20306,20306], 
                    'Wind':     [1439,1212,1136,1050,848,788,820,901,950,1096,1148,1361,1419,1680,1745,2030,2459,3235,3720,4238,5095,5627,5985,5800]}

In [ ]:
# convention: demand_t[t] = demand during hour t [MW]
demand_t = [65711,62818,61570,60688,61404,63945,69670,76579,80708,81355,82075,83116,82243,81289,79745,79214,81557,84042,82878,79834,75318,73229,71592,66178]

## 2. Build the LP model

In [ ]:
model = pyomo.ConcreteModel()

### 2.1 Define Sets

In [ ]:
model.T = pyomo.Set(initialize=T)
model.G = pyomo.Set(initialize=G)

### 2.2 Define Variables

In [ ]:
model.x_g_t = pyomo.Var(model.G, model.T, domain=pyomo.NonNegativeReals)    # dispatched power of technology g during hour t

### 2.3 Define Constraints

In [ ]:
def define_demand_restriction(model, t):
    return sum(model.x_g_t[g, t] for g in model.G) == demand_t[t]
model.demand_restriction = pyomo.Constraint(model.T, rule=define_demand_restriction)

In [ ]:
def define_capacity_restriciton(model, g, t):
    return model.x_g_t[g, t] <= availability_g_t[g][t]
model.capacity_restriciton = pyomo.Constraint(model.G, model.T, rule=define_capacity_restriciton)

### 2.4 Define Objective Function

In [ ]:
def define_objective_function(model):
    return sum(((tech_data_g[(g,'fuel price')] + tech_data_g[(g,'emission factor')] * CO2_PRICE) / tech_data_g[(g,'eta_el')]+tech_data_g[(g,'other variable costs')])*model.x_g_t[g,t] for g in model.G for t in model.T)
model.Obj = pyomo.Objective(rule=define_objective_function, sense=pyomo.minimize)

### 2.5 Write LP to File

In [ ]:
model.write('output/dispatch/04_esa_uebung_LP1_dispatch_hausaufgabe_loesung.lp', io_options={'symbolic_solver_labels':True})

### 2.6 Initialize the storage of dual variables of constraints

In [ ]:
model.dual = pyomo.Suffix(direction=pyomo.Suffix.IMPORT)

## 3. Solve the LP model

In [ ]:
optimizer = opt.SolverFactory('glpk')
solved_model = optimizer.solve(model, tee=True)

## 4. Get the results and statistics of the solved LP model

### 4.1 Print optimal objective value

In [ ]:
print("Optimal value: %.2f Mio. €" % (round(model.Obj.expr()/10**3,2)))

### 4.2 Print optimal dispatch path

In [ ]:
EPS = 1.e-6

In [ ]:
for g in G:
    for t in T:
        if pyomo.value(model.x_g_t[g,t]) > EPS:
            print("Dispatch %i TWh of technology %s in hour %s" % (pyomo.value(model.x_g_t[g,t]),g,t))

### 4.3 Print electricity prices as shadow prices of the demand constraint

In [ ]:
dual_values = pd.Series(list(model.dual.values()), index=pd.Index(list(model.dual.keys())))
electricity_shadow_prices = pd.Series(list(model.demand_restriction.values()), index=pd.Index(list(model.demand_restriction.keys()))).map(dual_values)

In [ ]:
for t in T:
    print("Electricity price during t= "+str(t)+": "+str(round(electricity_shadow_prices[t],1))+' €/MWh')

### 4.4 Calculate and print dark spread and clean spread

In [ ]:
marginal_costs_coal       = tech_data_g[('Coal','fuel price')]/tech_data_g[('Coal','eta_el')]+tech_data_g[('Coal','other variable costs')]
marginal_costs_coal_clean = marginal_costs_coal+tech_data_g[('Coal','emission factor')]*CO2_PRICE/tech_data_g[('Coal','eta_el')]

In [ ]:
dark_spread_t = {}
clean_spread_t= {}
for t in T:
    dark_spread_t[t]  = electricity_shadow_prices[t]-marginal_costs_coal   
    clean_spread_t[t] = electricity_shadow_prices[t]-marginal_costs_coal_clean

In [ ]:
for t in T:
    print('Dark  spread during t= '+str(t)+": "+str(round(dark_spread_t[t],1))+' €/MWh')

In [ ]:
for t in T:
    print('Clean spread during t= '+str(t)+": "+str(round(clean_spread_t[t],1))+' €/MWh')    